# Ghouls Submissions pt.2
The first notebook represents a nice analysis of the features etc. That was then broken out into a notebook where I tried a few models and submitted the results where The best results were 3. 1. 2.  In this notebook I really want to clean up the submissions with lessons learned - since I didnt have a complete understanding when I was generating the submissions then.

### Nested Cross Validation
The major focus of this work is nested cross valdiation - this was somewhat confusing and that is apparent in the code (i.e. there are some lines that simply look at how nested cross val is working). Ultimately there was a lot of learning here.

In [88]:
import pandas as pd
import numpy as np

In [89]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [90]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [91]:
train.columns

Index(['id', 'bone_length', 'rotting_flesh', 'hair_length', 'has_soul',
       'color', 'type'],
      dtype='object')

### Initially we dropped color 
Additionally we were mapping the outputs from strings to numbers, I don't think that is strictly necessary hence we are dropping that line below

In [92]:
# lets prep our features
dummies = pd.get_dummies(train['color'])
X = train.drop(['id','color','type'],axis=1)

# we also need to format our target variable 
# type_dict = {'Ghoul':0,'Goblin':1,'Ghost':2}
# Y = train.type.map(type_dict)

Y = train.type

## First Submission
**NOTE:** This is a nice way to grab the best model from our GridSearchCV object. I didn't use this for the actual first submission as it wasn't clear to me this is how you use it. **A more thorough approach would have been Nested Cross Validation**

In [93]:
pipe_rf = Pipeline([('poly', PolynomialFeatures()),
                    ('clf', RandomForestClassifier(random_state=1))])

param_grid = [{'poly__degree':[2,3],
               'clf__criterion':['gini','entropy'],
               'clf__max_features':[1,2,3,4,5,6,7,8,"sqrt","log2"],
               'clf__min_samples_split':[1,2,3,4,5,6,7]}]

gs = GridSearchCV(estimator=pipe_rf,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)

In [8]:
gs = gs.fit(X.values,Y.values)
print(gs.best_score_)
print(gs.best_params_)

0.762803234501
{'clf__min_samples_split': 1, 'clf__criterion': 'entropy', 'clf__max_features': 4, 'poly__degree': 2}


The best score doesnt really tell the right story - this should be an average with a standard deviation. Also this is NOT nested CV which is more robust

In [14]:
# since we already ran a few variants of clf above we can just grab our best model based on our best params
gs.best_estimator_.fit(X.values, Y.values)
y_pred = gs.best_estimator_.predict(test.drop(['id','color'],axis=1))

y_pred[:10]

array(['Ghoul', 'Goblin', 'Ghoul', 'Goblin', 'Ghost', 'Ghost', 'Ghoul',
       'Ghoul', 'Goblin', 'Ghoul'], dtype=object)

In [15]:
# create the submission
submission = pd.DataFrame({'id':test.id,'type':y_pred})
submission.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Goblin
4,13,Ghost


In [30]:
submission.to_csv('ghouls_01_alt.csv',index=False)

## Second Submission
This submission was based on a best_params_ from the previous spreadsheet - which probably wasn't validated properly. This submission received the lowest marks. 

    {'clf__criterion': 'gini', 'clf__min_samples_split': 5, 'clf__max_features': 4}

In [31]:
# # the first day GS came up with this: {'clf__criterion': 'gini', 'clf__min_samples_split': 5, 'clf__max_features': 4}
# clf = RandomForestClassifier(criterion='gini',min_samples_split=5,max_features=4)

# x = pd.get_dummies(train.drop(['id','type'],axis=1))
# y = train.type.map(type_dict)
# clf.fit(x.values, y)

# y_test = pd.get_dummies(test.drop(['id'],axis=1))
# y_pred2 = clf.predict(y_test.values)

In [32]:
# submission2 = pd.DataFrame({'id':test.id,'type':y_pred2})
# reverse_dict = {0:'Ghoul',1:'Goblin',2:'Ghost'}
# submission2['type'] = submission2.type.map(reverse_dict)

# submission2.head()

In [33]:
# # before we submit lets just make sure this isnt the same submission 2x
# submission.type == submission2.type

In [34]:
# submission2.to_csv('ghouls_02.csv',index=False)

## Submission 3
I decided to try the SVC model that appeared to run well the previous day. Once again pulling in the best params from the GridSearchCV I just randoml tried the following:

    {'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 10.0}
    
    SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
      max_iter=-1, probability=False, random_state=1, shrinking=True,
      tol=0.001, verbose=False)

In [35]:
# Fuck it lets try SVC {'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 10.0}
from sklearn.svm import SVC

In [36]:
def make_poly(df,n):
    """make a poly and add back in dummies"""
    poly = PolynomialFeatures(degree=n)
    if 'type' in df.columns:
        df = df.drop(['type'],axis=1)
    train_poly = poly.fit_transform(df.drop(['id','color'],axis=1).values)
    dummies = pd.get_dummies(df.color)
    t = pd.DataFrame(train_poly)
    return pd.concat([t,dummies],axis=1)

In [37]:
clf = SVC(kernel='rbf',gamma=0.01,C=10,random_state=1)

clf.fit(make_poly(train,3).values, train.type.values)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

In [38]:
y_pred3 = clf.predict(make_poly(test,3).values)

In [39]:
y_pred3[:10]

array(['Ghoul', 'Goblin', 'Ghoul', 'Ghost', 'Ghost', 'Ghost', 'Ghoul',
       'Ghoul', 'Goblin', 'Ghoul'], dtype=object)

In [40]:
submission3 = pd.DataFrame({'id':test.id,'type':y_pred3})
submission3.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Ghost
4,13,Ghost


In [121]:
# submission3.to_csv('ghouls_03.csv',index=False)

## Let's see if we can't clean this up some
We could have evaluated our models with nested grid search to see if the hyper parameters changed
Also since I'm starting to get the hang of pipelines we could have tried a bunch more tuned models and seen which is
best

In [59]:
# quickly lets just check-in on our data - here our training data has no color attribute any more
X.head()

,bone_length,rotting_flesh,hair_length,has_soul
0,0.354512,0.350839,0.465761,0.781142
1,0.575560,0.425868,0.531401,0.439899
2,0.467875,0.354330,0.811616,0.791225
3,0.776652,0.508723,0.636766,0.884464
4,0.566117,0.875862,0.418594,0.636438


In [60]:
# note our test data in untouched
test.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color
0,3,0.471774,0.387937,0.706087,0.698537,black
1,6,0.427332,0.645024,0.565558,0.451462,white
2,9,0.549602,0.491931,0.660387,0.449809,black
3,10,0.638095,0.682867,0.471409,0.356924,white
4,13,0.361762,0.583997,0.377256,0.276364,black


### Nested Cross Validation

In [94]:
pipe_svc = Pipeline([ #('poly',PolynomialFeatures(degree=3)), #removing this so we can use make_poly
                      ('clf',SVC(random_state=1))])

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
param_grid = [{'clf__C': param_range,
               'clf__kernel': ['linear']},
              {'clf__C': param_range,
               'clf__gamma': param_range,
               'clf__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv=10,
                 n_jobs=-1)

In [95]:
# here is the nested part - an unbiased estimate of how well the model will perform on new data
scores = cross_val_score(gs,make_poly(train,3).values,train.type.values, scoring='accuracy',cv=5)
print(scores)
print('Accuracy: {0:.3f} +/- {1:.3f}'.format(np.mean(scores),np.std(scores)))

[ 0.74666667  0.73333333  0.81081081  0.64864865  0.67123288]
Accuracy: 0.722 +/- 0.058


In [96]:
# this approach is excellent for getting the scores but it doesn't give the best model parameters
# https://rvraghav93.blogspot.com/2015/06/gsoc-2015-psf-scikit-learn-nested-cross.html
# unfortunately we have to nest the two loops somewhat by hand to see the individual parameters
from sklearn.cross_validation import StratifiedShuffleSplit

cv_outer = StratifiedShuffleSplit(train.type.values, n_iter=10,test_size=0.3, random_state=1)

for training_set_indices_i, testing_set_indices_i in cv_outer:
    training_set_i = make_poly(train,3).values[training_set_indices_i], train.type.values[training_set_indices_i]
    testing_set_i = make_poly(train,3).values[testing_set_indices_i], train.type.values[testing_set_indices_i]
    gs.fit(*training_set_i)
    print(gs.best_params_, '\t\t', gs.score(*testing_set_i))


{'clf__kernel': 'rbf', 'clf__gamma': 0.001, 'clf__C': 100.0} 		 0.732142857143
{'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 100.0} 		 0.705357142857
{'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.767857142857
{'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.794642857143
{'clf__kernel': 'linear', 'clf__C': 1.0} 		 0.732142857143
{'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.776785714286
{'clf__kernel': 'rbf', 'clf__gamma': 1.0, 'clf__C': 1.0} 		 0.660714285714
{'clf__kernel': 'rbf', 'clf__gamma': 0.001, 'clf__C': 100.0} 		 0.732142857143
{'clf__kernel': 'linear', 'clf__C': 100.0} 		 0.705357142857
{'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.714285714286


### How this is actually working...
If you look above there are quite a few values... this is what a run looks like with only 5 training loops:

    cv_outer = StratifiedShuffleSplit(train.type.values, n_iter=5,test_size=0.3, random_state=1)

    for training_set_indices_i, testing_set_indices_i in cv_outer:
        training_set_i = make_poly(train,3).values[training_set_indices_i], train.type.values[training_set_indices_i]
        testing_set_i = make_poly(train,3).values[testing_set_indices_i], train.type.values[testing_set_indices_i]
        gs.fit(*training_set_i)
        print(gs.best_params_, '\t\t', gs.score(*testing_set_i))


    {'clf__kernel': 'rbf', 'clf__gamma': 0.001, 'clf__C': 100.0} 		 0.732142857143
    {'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 100.0} 		 0.705357142857
    {'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.767857142857
    {'clf__kernel': 'linear', 'clf__C': 10.0} 		 0.794642857143
    {'clf__kernel': 'linear', 'clf__C': 1.0} 		 0.732142857143

This isn't quite working the way I thought it might... this is presenting the best estimator for a seemingly random modeling of the data. **I need to better understand this apparent randomness i.e. where it is coming from** But what we see is that we could rank the number of times a model is the "best". We see from above that a few models are frequently top performers... BUT most notably 

    {'clf__kernel': 'linear', 'clf__C': 10.0}
    
Which is surprising since our best scoring model is fit with these params (which also show up in our list - but not nearly as frequently as that of the above value

    {'clf__kernel': 'rbf', 'clf__gamma': 0.01, 'clf__C': 10.0}

#### Lets see how the linear SVC model works in a submission

In [83]:
# so this is interesting, the best parameters are somewhat different than the last GridSearch
# this is likely the result of a difference in the settings - the good news is that we have a 
# solid bench mark for performance

# lets see how this slight variation of the model performs as a Kaggle submission
# first we set our "best parameters" from above
pipe_svc.set_params(**{'clf__kernel': 'linear', 'clf__C': 10.0})
pipe_svc.fit(make_poly(train,3).values, train.type.values)
# then we can predict the test set
y_pred4 = pipe_svc.predict(make_poly(test,3).values)

In [85]:
# it looks like 44 submissions are different
np.bincount(y_pred4 == y_pred3)

array([ 44, 485])

In [86]:
# lets see how this performs in Kaggle
submission4 = pd.DataFrame({'id':test.id.values,'type':y_pred4})
submission4.head()

,id,type
0,3,Ghoul
1,6,Goblin
2,9,Ghoul
3,10,Goblin
4,13,Ghost


In [87]:
submission4.to_csv('ghouls_04.csv',index=False)

#### Kaggle Results
This submission is worse than the previous model which was much simpler. .72 vs .74 (it is also worse than the same model as submission 3 with no color values .73) The previous model simply ran a Grid Search to figure out the hyperparameters then performed minimal feature engineering then generated a prediction. **It looks like the major take away is that nested cross validation is excellent for model comparison - but NOT for hyper-parameter selection**